# 配置环境

在运行之前需要安装所需的python包。建议使用anaconda创建一个新的虚拟环境，具体方法为打开Anaconda navigator，创建环境，环境名称可以命名为ByteBites。

激活环境，确保在vscode的资源管理器中打开ByteBites文件夹。在vscode的左侧python扩展中，在global environments，选中刚才安装的环境，点击open in terminal，在打开的命令行中输入以下命令安装所需的包：
```bash
pip install -r requirements.txt
```

In [1]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

DATASET_PATH = r"restaurant_all.csv"

### 🗂️ 数据检索与特征解析提示词
系统将根据你的定位与需求，从本地数据库检索店铺信息，分析评论、标签、菜单等多维特征，为智能匹配和个性推荐做准备。


# 导入CSV数据库

以下代码从CSV文件导入餐厅数据，并生成一个新的文档对象documents。这个功能由DataFrameLoader提供。

### 数据结构说明

documents可以理解为一个list表格，每一行是包含两个字段的dict，两个字段分别为：  
- `page_content`：值为字符串。将餐厅的所有信息整合在一个字符串中，用于喂给大模型。
- `metadata`：值为dict。储存准备用于初过滤的硬性指标值，如经纬度、营业时间等。 

### 示例

假设原始csv文件如下：
```csv
name,address,location,type,tel,cost,rating,opentime_today,opentime_week,tag
巴蜀鱼花(南大店),湖南路街道汉口路30号,"118.779220,32.053685",餐饮服务;中餐厅;火锅店,15380870767,,4.4,09:00-21:00,周一至周日 09:00-21:00,
陕老顺肉夹馍,汉口路30号,"118.779105,32.053716",餐饮服务;餐饮相关场所;餐饮相关,15924140124,,4.4,10:00-21:00,周一至周日 10:00-21:00,肉夹馍
```

导入后的结果为：
```python
documents = [
    ..., # 第一行略
    {
        "page_content": "name=陕老顺肉夹馍\naddress=汉口路30号\nlocation=118.779105,32.053716\ntype=餐饮服务;餐饮相关场所;餐饮相关\ntag=肉夹馍\nrating=4.4\nopentime_today=10:00-21:00\nopentime_week=周一至周日 10:00-21:00\ntel=15924140124",
        "metadata": {
            "location": "118.779105,32.053716",
            "opentime_week": "周一至周日 10:00-21:00"
        }
    }
]
```

### **注意**

- 读取店铺数据的代码更改，导致csv文件中字段发生变化时：def content_func 中 python content_fields 的值要随之更改。
- 想作为硬性指标的字段发生变化时：调用部分 metadata_fields 中的值要更改。

### ⭐ 推荐结果卡片展示与解释提示词
为你推荐多家餐厅，并说明推荐理由。你可选择“喜欢/一般/不喜欢”，或指出需要排除/优先的类型。


In [6]:
def get_documents(content_func=lambda row: row['name'] + '\n' + row['tag'],
                  metadata_fields=[]):
    """
    加载并处理餐厅数据，生成文档对象
    
    参数:
        content_func: 处理page_content的函数
        metadata_fields: 需要包含在metadata中的字段列表
        
    返回:
        文档对象列表，包含page_content和metadata
    """
    # 加载数据库
    dataset_df = pd.read_csv(DATASET_PATH)
    dataset_df.drop_duplicates(inplace=True)
    
    # 处理内容
    dataset_df['page_content'] = dataset_df.apply(content_func, axis=1)
    
    # 确保page_content在metadata中
    metadata_fields = list(set(metadata_fields + ['page_content']))
    
    # 使用DataFrameLoader生成文档对象
    loader = DataFrameLoader(dataset_df[metadata_fields], page_content_column='page_content')
    return loader.load()

def content_func(row) -> str:
    """
    生成每家店铺的完整信息字符串
    
    参数:
        row: 数据行
        
    返回:
        包含店铺所有信息的字符串
    """
    # 基础信息字段
    content_fields = [
        "name", "address", "type", "tag", 
        "cost", "rating", "opentime_today", "opentime_week"
    ]
    
    # 新增的多维评价字段
    rating_fields = [
        "dp_rating", "dp_taste_rating", "dp_env_rating",
        "dp_service_rating", "dp_comment_num"
    ]
    
    # 推荐菜和评论相关字段
    comment_fields = [
        "dp_recommendation_dish", "dp_comment_keywords",
        "dp_top3_comments"
    ]
    
    # 生成信息字符串
    info_parts = []
    
    # 添加基础信息
    for field in content_fields:
        if pd.notna(row[field]):
            info_parts.append(f"{field}={row[field]}")
    
    # 添加评分信息
    rating_info = []
    for field in rating_fields:
        if pd.notna(row[field]):
            rating_info.append(f"{field}={row[field]}")
    if rating_info:
        info_parts.append("评分信息:\n" + "\n".join(rating_info))
    
    # 添加推荐菜和评论
    if pd.notna(row.get("dp_recommendation_dish")):
        info_parts.append(f"推荐菜: {row['dp_recommendation_dish']}")
    
    if pd.notna(row.get("dp_comment_keywords")):
        info_parts.append(f"评论关键词: {row['dp_comment_keywords']}")
    
    if pd.notna(row.get("dp_top3_comments")):
        info_parts.append("精选评论:\n" + row['dp_top3_comments'].replace("|", "\n"))
    
    return '\n'.join(info_parts)

# 调用 - 增加新字段到metadata中
metadata_fields = [
    "location", "opentime_week",
    "dp_rating", "dp_taste_rating", "dp_env_rating",
    "dp_service_rating", "dp_comment_num"
]

documents = get_documents(content_func, metadata_fields=metadata_fields)

# 展示更新后的文档结构
print("=== 更新后的文档示例 ===")
print(documents[2].page_content)
print("\n=== 元数据 ===")
print(documents[2].metadata)

=== 更新后的文档示例 ===
name=同廣鸣港式烧腊(南京大学店)
address=汉口路47-2号
type=餐饮服务;中餐厅;中餐厅
tag=港式烧腊
rating=4.4
opentime_today=10:00-22:00
opentime_week=周一至周日 10:00-22:00
评分信息:
dp_rating=3.5
dp_taste_rating=3.5
dp_env_rating=3.7
dp_service_rating=3.6
dp_comment_num=34.0
推荐菜: 自选双拼饭,白灼青菜,深井烧鵝,蜜汁叉烧饭,鹅翅,白切鸡,深井烧鹅饭,蜜汁叉烧,香港油鸡饭,澳门烧肉饭
评论关键词: {"味道赞: 6", "性价比高: 6", "服务热情: 3", "环境一般: 3", "午餐: 3"}
精选评论:
[("2025-05-22", "只能说这烧腊的味道在两广开店活不过一个月，骗骗南京人罢了"), ("2025-05-22", "这家港式腊烧店开在南大鼓楼校区门口的汉口路上，距离南京大学很近，来这边吃饭的以学生居多，到了饭店人挺多的。口味：点了他们家的三拼腊烧饭，叉烧，烧鸭，白斩鸡，叉烧：鲜嫩多汁，微微带点甜，非常好吃，烧鸭：经典的广式烧鸭，味道也不错，白斩鸡：白斩鸡味道有点淡，并不是我想象中的白斩鸡，鸡肉还可以，所有的配菜都是现切现剁的，还是很新鲜的..."), ("2025-05-18", "南大门口，门头不大，口味很好，份量很足，服务很好。")]

=== 元数据 ===
{'location': '118.779262,32.053522', 'dp_comment_num': 34.0, 'dp_taste_rating': 3.5, 'dp_service_rating': 3.6, 'dp_env_rating': 3.7, 'dp_rating': 3.5, 'opentime_week': '周一至周日 10:00-22:00'}


In [ ]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings
# import os

# #加载环境变量
# load_dotenv()
# #配置嵌入模型
# EMBEDDING_MODEL_NAME = "text-embedding-3-small"  
# embedding_model = OpenAIEmbeddings(
#     model=EMBEDDING_MODEL_NAME,
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

# 配置嵌入vecterbase所用的模型

用的是huggingface上面的某个轻量级开源模型。

In [ ]:
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# import os
# import numpy as np

# # 初始化 HuggingFaceEmbeddings 模型，调整以支持多轮对话
# embedding_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={"device": "cpu"},
#     encode_kwargs={"normalize_embeddings": True, "batch_size": 16}  # 增加批量处理支持
# )

# # 示例：生成多轮对话的嵌入向量
# dialogue_history = [
#     "用户：有没有四川的面馆？",
#     "助手：有一家叫西安特色面馆的店，虽然是陕西菜，但也有类似的面食。",
#     "用户：还有其他推荐吗？"
# ]

# # 将对话历史拼接为单个字符串
# dialogue_context = "\n".join(dialogue_history)
# result = embedding_model.embed_query(dialogue_context)

# # 转换为 numpy 数组并打印信息
# array = np.array(result)
# print(f"embedding shape: {array.shape}\nembedding norm: {np.linalg.norm(array, ord=2)}")

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import numpy as np

# 本地部署的中文向量模型（BGE 小型版），效果在中文检索/推荐场景中优异
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh",              # 中文专用小模型
    model_kwargs={"device": "cpu"},               # 如果有英伟达GPU改为 "cuda" 
    encode_kwargs={
        "normalize_embeddings": True,             # 归一化向量有助于相似度计算
        "batch_size": 16                          # 根据硬件调整批量大小
    }
)

# 示例：生成多轮对话的嵌入向量
dialogue_history = [
    "用户：有没有四川的面馆？",
    "助手：有一家叫西安特色面馆的店，虽然是陕西菜，但也有类似的面食。",
    "用户：还有其他推荐吗？"
]
dialogue_context = "\n".join(dialogue_history)
result = embedding_model.embed_query(dialogue_context)

array = np.array(result)
print(f"embedding shape: {array.shape}, norm: {np.linalg.norm(array):.4f}")

embedding shape: (512,), norm: 1.0000


# FAISS数据库

在代码 FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" 中，"faiss_index_cosine" 是 FAISS 向量索引的本地存储路径，保存的是经过向量化处理后的文档索引数据。以下是详细解释：

存储的具体内容
当调用 vector_db.save_local(FAISS_REVIEWS_PATH_COSINE) 时，会在该路径下生成以下文件：

index.faiss
二进制文件，存储向量索引的核心数据（包括向量数据、索引结构等）。

index.pkl（可选）
存储元数据（如文档的原始文本、ID等，需通过 LangChain 额外配置）。

这些文件共同构成一个完整的可复用的向量数据库。

In [9]:
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" # 向量库存储路径
FAISS_INDEX_NAME = "index" # 向量库索引名称
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE" # 向量库距离计算策略

# 用于根据csv数据生成向量库的函数。documents就是前面csv数据的导入结果。embedding_model就是上面定义的嵌入模型。
def get_vector_database(documents, embedding_model, distance_strategy):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= distance_strategy
      )
  return vector_database

# 嵌入向量库。分批次处理文档，加入了等待时间，避免API限制（现在的版本是把向量库保存在本地，没有对应限制）。
import time
doclen = len(documents) # 这里的长度指的是前面读的数据的行数。
for batch in range(doclen//100 + 1): # 将每个店铺的信息独立转换为一个向量，且每次并行处理 100 个店铺的向量
    docs = documents[batch*100:(batch+1)*100]
    if batch == 0:
        vector_db = get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE)
    else:
        vector_db.merge_from(get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE))
    time.sleep(10) # 每次处理完100条数据，休眠10秒，防止api限制。
    
#储存并加载向量库
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH_COSINE, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_COSINE,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             allow_dangerous_deserialization=True) # 允许反序列化

### 验证效果

In [12]:
docs = vector_db.similarity_search("披萨", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=比萨时光(上海路店)
address=上海路81-7号
type=餐饮服务;快餐厅;快餐厅
tag=提拉米苏,水果披萨,帕尔玛进口风干火煺披萨,榴莲芝士披萨,意大利肉酱面,孜然烤羊排,水牛奶酪沙拉,海鲜岛粉丝沙律,培根披萨,榴莲芝士全肉双拼披萨,土豆泥,比萨
cost=72.0
rating=4.3
opentime_week=周一，周三至周日 11:00-14:00，16:00-21:00
评分信息:
dp_rating=4.4
dp_taste_rating=4.4
dp_env_rating=4.3
dp_service_rating=4.4
dp_comment_num=591.0
推荐菜: 榴莲芝士全肉双拼披萨,泰式咖喱大虾,四喜团圆披萨,招牌香辣肉碎饭,菲力牛排沙律,小食拼盘,超级全肉披萨,奶油蘑菇汤,榴莲披萨
评论关键词: {"味道赞": 151, "牛肉赞": 10, "主食赞": 10, "海鲜棒": 9, "上菜快": 4, "朋友聚餐": 10, "现做现卖": 10, "约会圣地": 9, "文艺清新": 6, "闺蜜聚会": 6}
精选评论:
[("2025-05-14", "上海路那边一家主打披萨🍕和西式快餐的店铺，店铺不大，环境方面还不错。\n 中午去的时候，店里顾客还挺多的，看起来生意不错。\n 他家的几款招牌披萨🍕「榴莲芝士全肉双拼披萨」「超级全肉披萨」「四喜团圆披萨」「阿尔巴培根披萨」「帕尔玛进口风干火腿披萨」「金枪鱼披萨」「水果披萨」「玛格丽特 pizza」等等，比较喜欢他家的招牌榴莲披萨，味道吃起来甜而不腻哦！"), ("2025-03-30", "上海路附近的一家披萨店铺，店铺不大，环境还可以吧！他家的招牌披萨还挺有特色的，吃起来味道还不错。他家的泰式咖喱大虾🍤 咖喱味道还挺浓郁的，意面🍝也挺好吃的。他家的招牌油炸小吃也还不错，还会再来光顾的哦！"), ("2025-03-29", "饭点时间去的，店里的人还挺多的，看起来生意不错。点了几道他家的招牌菜品，看起来都还可以。他家的招牌披萨味道不错，披萨上面的馅料放的还挺多的，口感也比较松软哦！他家的招牌小吃也是炸的香酥可口，还会回购的呢！")]' metadata={'location': '118.774805,32.052207', '

In [19]:
docs = vector_db.similarity_search("烤鸭", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=南京沉香鸭馆(汉口西路店)
address=汉口西路1号
type=餐饮服务;中餐厅;中餐厅
rating=4.3' metadata={'location': '118.773660,32.054823', 'dp_comment_num': nan, 'dp_taste_rating': nan, 'dp_service_rating': nan, 'dp_env_rating': nan, 'dp_rating': nan, 'opentime_week': nan}

page_content='name=北京烤鸭
address=南秀大厦北门旁
type=餐饮服务;餐饮相关场所;餐饮相关
tag=北京烤鸭
cost=106.0
rating=4.3
opentime_today=08:00-22:00
opentime_week=周一至周日 08:00-22:00' metadata={'location': '118.773471,32.054819', 'dp_comment_num': nan, 'dp_taste_rating': nan, 'dp_service_rating': nan, 'dp_env_rating': nan, 'dp_rating': nan, 'opentime_week': '周一至周日 08:00-22:00'}

page_content='name=徐家鸭子店(丹凤街店)
address=丹凤街17号-6(珠江路地铁站3号口步行490米)
type=餐饮服务;快餐厅;快餐厅
tag=鸭爪,鸭翅,盐水鸭,南京烤鸭,卤菜,烤鸭,鸭肉
rating=4.7
opentime_today=08:00-20:00
opentime_week=周一至周日 08:00-20:00' metadata={'location': '118.787041,32.053462', 'dp_comment_num': nan, 'dp_taste_rating': nan, 'dp_service_rating': nan, 'dp_env_rating': nan, 'dp_rating': nan, 'opentime_week': '周一至周日 08:00-20:00'}

page_cont

# 配置大语言模型
需要在ByteBites目录下创建配置文件.env，里面添加大语言模型的配置信息。密钥要自己申请。代码支持的模型包括：OpenAI、Deepseek、通义千问等。

.env文件形如：

```python
# OpenAI
OPENAI_API_KEY = 
OPENAI_MODEL_NAME = "gpt-4o-mini"

# Deepseek
DEEPSEEK_API_KEY = 
DEEPSEEK_BASE_URL = 'https://api.deepseek.com'
DEEPSEEK_MODEL_V3 = 'deepseek-chat'
DEEPSEEK_MODEL_R1 = 'deepseek-reasoner'
```

In [20]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import getpass

# 载入本地 .env 文件
load_dotenv()

# 设置 DeepSeek API KEY
if not os.environ.get("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek-AI: ")

# 初始化 LangChain 的 ChatOpenAI（适配 DeepSeek）
llm = ChatOpenAI(
    openai_api_key=os.environ["DEEPSEEK_API_KEY"],
    openai_api_base="https://api.deepseek.com/v1",
    model_name="deepseek-reasoner",  # 或者 deepseek-v3，具体根据官方命名
    temperature=0.7
)

# 示例对话调用
response = llm([HumanMessage(content="你好，请用一句话介绍你自己")])
print(response.content)

C:\Users\Chrom\AppData\Local\Temp\ipykernel_23020\3020143093.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
C:\Users\Chrom\AppData\Local\Temp\ipykernel_23020\3020143093.py:23: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content="你好，请用一句话介绍你自己")])


你好呀！我是一位专业实用又温暖贴心的智能助手，随时准备为你解答问题、提供建议或陪你聊天～😊


In [21]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
messages = [
    SystemMessage("Translate the following from English into French"),
    HumanMessage("hi!"),
]
llm.invoke(messages)

AIMessage(content='Salut !', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 14, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 149, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 14}, 'model_name': 'deepseek-reasoner', 'system_fingerprint': 'fp_393bca965e_prod0425fp8', 'finish_reason': 'stop', 'logprobs': None}, id='run-c579805d-7f9c-4a95-a594-b61f8c4f73d3-0')

# 用LangChain包设置和大模型交互的工作流

In [18]:
def create_user_preference(
    environment=3, 
    taste=4,
    service=3,
    cost_performance=4,
    hygiene=4,
    health=3,
    waiting_time=2,
    distance=3,
    preferred_cuisines=["川菜", "火锅"],
    disliked_cuisines=["日料"],
    budget_range=[50, 150],
    special_requirements="无"
):
    """
    创建用户偏好字典
    """
    return {
        "环境": environment,
        "口味": taste,
        "服务": service,
        "性价比": cost_performance,
        "卫生": hygiene,
        "营养健康": health,
        "排队时间": waiting_time,
        "距离": distance,
        "偏好菜系": preferred_cuisines,
        "不喜欢的菜系": disliked_cuisines,
        "预算范围": budget_range,
        "特殊要求": special_requirements
    }

system_prompt_template = """
# 你的角色

你是“菜根探”——一名智能美食推荐助手。你的工作是根据用户的个性化偏好和实时需求，从数据库中推荐最优餐厅。

# 1. 用户个人偏好

用户的个人偏好如下（已由前端页面采集）：
'''
{user_preference}
'''
- 偏好指标包括：环境、口味、服务、性价比、卫生、营养健康、排队时间、距离等。
- 每项得分为0-5分，数值越高代表用户在挑选餐厅时对该项要求越高，指标越靠前说明越重要。
- 请结合这些分值，为后续餐厅筛选与加权打分分配不同权重，优先满足得分高和排名靠前的指标项。
- 偏好含义参考：  
    - 环境：重视餐厅环境、氛围、安静度、空间舒适度  
    - 口味：重视菜品风味、辣度、甜度、食材正宗性  
    - 服务：重视服务态度、效率、人员素质  
    - 性价比：追求花更少的钱享受更好服务和菜品  
    - 卫生：对用餐卫生和食材安全的重视  
    - 营养健康：追求健康、低油低盐、营养搭配  
    - 排队时间：关注是否需等位、出餐速度  
    - 距离：优先考虑距离近、交通便利的餐厅
    请特别注意以下关键信息：
    - 各维度评分(0-5分): {preference_scores}
    - 偏好菜系: {preferred_cuisines}
    - 不喜欢的菜系: {disliked_cuisines}
    - 预算范围: {budget_range}元
    - 特殊要求: {special_requirements}

    # 2. 推荐策略
    - 优先匹配用户评分高的维度(4-5分)
    - 确保推荐餐厅在用户预算范围内
    - 避免推荐用户不喜欢的菜系
    - 考虑用户特殊要求
# 2. 用户实时需求和场景采集

-用户会用自然语言表达本次用餐需求（如预算、人数、距离、期望场景/菜系/口味等），你需要智能解析这些内容，将其与个人偏好结合起来，为用户进行匹配推荐。
-鼓励用户用自然语言表达当前用餐需求（如预算、人数、距离、类型、菜系、期望场景/氛围等），支持用户自由描述，也可追问补全关键信息。
-你需解析这些需求，明确硬性（预算/营业/距离/人数/时段）与软性（口味/氛围/健康/环境）约束。

# 3. 餐厅数据库字段说明

你所参考的数据库中，每家餐厅包含以下字段：
- name（餐厅名称）
- dp_cost（大众点评人均消费）
- dp_rating（大众点评综合评分）
- dp_taste_rating（大众点评口味评分）
- dp_env_rating（大众点评环境评分）
- dp_service_rating（大众点评服务评分）
- dp_comment_num（大众点评评论数）
- dp_recommendation_dish（推荐菜品）
- dp_comment_keywords（高频评论关键词）
- dp_top3_comments（精选评论）
- address（门店地址）
- location（地理坐标或位置描述）
- type（餐厅类型/菜系）
- tel（联系电话）
- cost（其他平台人均消费）
- rating（平台综合评分）
- opentime_today（今日营业时间）
- opentime_week（每周营业时间）
- tag（标签/特色）
请注意以上信息可能会有确实或不完整的情况，需要你再结合网络搜索给出更加全面客观的结果，如果没有获取到清晰明确的信息可忽略该维度，不要输出虚构内容。

# 4. 推荐输出要求
每轮推荐请严格输出：
- 按“个人偏好+实时需求”综合得分排序，推荐3~5家最优餐厅。
- 每家餐厅详细输出：
    1. 基础信息：名称、地址、类型/菜系、标签/特色、联系电话、人均消费、营业时间、距离等
    2. 多维匹配打分：综合得分（0-100分/5分制），并对各主维度（环境、口味、服务、性价比等）单独打分，清晰展示与用户偏好对应分值，并说明原因
    3. AI生成个性化推荐理由：**结合用户高权重指标具体展开**，如“该店环境评分4.8，安静优雅，极度适合你对环境的高要求”，并适当引用精选评论、推荐菜、关键词
    4. 最佳交通方式与所需时间：请根据用户出发地（如未输入，请主动引导用户补充自己的具体位置），为每家餐厅给出推荐的交通方式（步行、地铁、打车、公交等）与大致所需时间（如步行10分钟、地铁2站共20分钟等），提升实际可达性与用户体验。
    5. 用户补充引导：主动建议用户进一步细化需求（如预算、口味、场景、特殊要求、出发地等），如果推荐未完全匹配高分项，请提示可补充更多限制条件以优化结果。特别是在未获得用户具体位置时，提醒其补充出发地以优化路线推荐。
# 补充说明
- 如果用户未提供出发地，请在回复末尾礼貌建议：“为获得更准确的交通方案，请告知你当前或希望出发的具体位置”。
- 交通方案可根据餐厅地址和用户位置生成（如无法获取实际路线，可用“预计步行约X分钟”格式估算）。


# 规则
- 你只能基于数据库现有信息推荐，不允许虚构不存在的餐厅。
- 如某项信息缺失，请如实说明“该项暂无数据”。
- 非餐饮、无关问题请委婉回复“仅能为您提供美食/餐厅推荐服务”。
- 推荐内容需结构化、条理清晰、易于用户理解和决策。


# 当前对话历史
{history}

# 数据库内容
{context}
"""

def format_user_preference(user_pref):
    """格式化用户偏好为字符串"""
    scores = "\n".join([f"{k}: {v}分" for k,v in user_pref.items() if isinstance(v, int)])
    # 健壮性处理预算范围
    budget_range = user_pref.get('预算范围', [])
    if isinstance(budget_range, list) and len(budget_range) >= 2:
        budget_str = f"{budget_range[0]}-{budget_range[1]}元"
    elif isinstance(budget_range, list) and len(budget_range) == 1:
        budget_str = f"{budget_range[0]}元"
    else:
        budget_str = "未填写"
    return (
        f"用户偏好评分:\n{scores}\n\n"
        f"偏好菜系: {', '.join(user_pref.get('偏好菜系', []))}\n"
        f"不喜欢的菜系: {', '.join(user_pref.get('不喜欢的菜系', []))}\n"
        f"预算范围: {budget_str}\n"
        f"特殊要求: {user_pref.get('特殊要求', '无')}"
    )

def extract_preference_vars(user_pref):
    # preference_scores: 环境、口味、服务、性价比、卫生、营养健康、排队时间、距离
    scores = []
    for k in ["环境", "口味", "服务", "性价比", "卫生", "营养健康", "排队时间", "距离"]:
        if k in user_pref:
            scores.append(f"{k}: {user_pref[k]}分")
    return {
        "preference_scores": "\n".join(scores),
        "preferred_cuisines": ", ".join(user_pref.get("偏好菜系", [])),
        "disliked_cuisines": ", ".join(user_pref.get("不喜欢的菜系", [])),
        "budget_range": f"{user_pref.get('预算范围', [0,0])[0]}-{user_pref.get('预算范围', [0,0])[1]}元",
        "special_requirements": user_pref.get("特殊要求", "无"),
    }

In [22]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableMap, RunnableLambda
from langchain.memory import ConversationBufferMemory

# 假设 vector_db 和 llm 已提前定义
# vector_db = FAISS.load_local(...)
# llm = ...

# 初始化对话历史记忆模块
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

human_prompt_template = """
{question}
user_preference={user_preference}
"""

# 使用langchain自带的prompts模板编辑器来编辑prompt格式
system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["history", "context"], template=system_prompt_template
    )
)
human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template=human_prompt_template)
)
messages = [system_prompt, human_prompt]
total_prompt_template = ChatPromptTemplate(
    input_variables=[
        "history", "context", "question", "user_preference",
        "preference_scores", "preferred_cuisines", "disliked_cuisines", "budget_range", "special_requirements"
    ],
    messages=messages,
)

# 配置RAG的初步检索器
# 修正：确保只将 question 字段传递给 retriever
reviews_retriever = RunnableLambda(lambda x: x["question"]) | vector_db.as_retriever(search_kwargs={'k': 20,})

# 配置和大模型交互的完整chatbot，加入历史记忆
# 确保 user_preference 已定义（如未定义则初始化为空字典或采集用户画像）
try:
    user_preference
except NameError:
    user_preference = {}

review_chain = (
    RunnableMap({
        "history": RunnablePassthrough(lambda _: memory.load_memory_variables({})),
        "context": reviews_retriever,  # 修正后 context 只传递 question 字段
        "question": RunnablePassthrough(),
        "user_preference": lambda x: format_user_preference(user_preference),
        "preference_scores": lambda x: extract_preference_vars(user_preference)["preference_scores"],
        "preferred_cuisines": lambda x: extract_preference_vars(user_preference)["preferred_cuisines"],
        "disliked_cuisines": lambda x: extract_preference_vars(user_preference)["disliked_cuisines"],
        "budget_range": lambda x: extract_preference_vars(user_preference)["budget_range"],
        "special_requirements": lambda x: extract_preference_vars(user_preference)["special_requirements"],
    })
    | total_prompt_template
    | llm
    | StrOutputParser()
)

# === 用户画像采集与转换 ===
def collect_user_profile():
    """
    交互式采集用户画像信息，并返回统一格式的用户偏好字典
    """
    print("请根据提示输入你的用餐偏好（1-5分，5分最重视）")
    def ask_score(q):
        while True:
            try:
                s = int(input(q))
                if 1 <= s <= 5:
                    return s
                else:
                    print("请输入1-5之间的整数")
            except:
                print("请输入数字")
    cost_performance = ask_score("请用1-5分评价你对‘性价比’的重视程度（1-5分）：")
    hygiene = ask_score("请用1-5分评价你对‘卫生’的重视程度（1-5分）：")
    taste = ask_score("请用1-5分评价你对‘口味’的重视程度（1-5分）：")
    service = ask_score("请用1-5分评价你对‘服务’的重视程度（1-5分）：")
    waiting_time = ask_score("请用1-5分评价你对‘排队时间’的重视程度（1-5分）：")
    health = ask_score("请用1-5分评价你对‘营养健康’的重视程度（1-5分）：")
    environment = ask_score("请用1-5分评价你对‘环境氛围’的重视程度（1-5分）：")
    distance = ask_score("请用1-5分评价你对‘距离远近’的重视程度（1-5分）：")
    preferred_cuisines = input("你偏好的菜系有哪些？（用逗号分隔，如川菜,火锅，可跳过）：").split(",") if input("是否有偏好菜系？(y/n):").lower() == "y" else []
    disliked_cuisines = input("你不喜欢的菜系有哪些？（用逗号分隔，如日料，可跳过）：").split(",") if input("是否有不喜欢的菜系？(y/n):").lower() == "y" else []
    budget_str = input("你的预算范围是多少？（如30-80，单位元）：")
    try:
        budget_range = [int(x) for x in budget_str.split("-")]
    except:
        budget_range = [0, 999]
    special_requirements = input("你还有什么特别的饮食偏好、忌口、禁忌或用餐习惯需要补充吗？（如无可跳过）：") or "无"
    # 统一格式
    user_pref = {
        "环境": environment,
        "口味": taste,
        "服务": service,
        "性价比": cost_performance,
        "卫生": hygiene,
        "营养健康": health,
        "排队时间": waiting_time,
        "距离": distance,
        "偏好菜系": [c.strip() for c in preferred_cuisines if c.strip()],
        "不喜欢的菜系": [c.strip() for c in disliked_cuisines if c.strip()],
        "预算范围": budget_range,
        "特殊要求": special_requirements
    }
    print("【你的用户画像已生成】\n", user_pref)
    return user_pref

# === 主流程示例 ===
def main():
    print("欢迎使用智能美食推荐系统！")
    # 1. 采集用户画像
    user_profile = collect_user_profile()
    # 2. 进入推荐问答循环
    from langchain.memory import ConversationBufferMemory
    memory = ConversationBufferMemory(memory_key="history", return_messages=True)
    print("\n请输入你的本次用餐需求（如‘预算50元内，想吃辣的，适合2人，距离不远’）：")
    while True:
        question = input("你的用餐需求/问题：")
        if not question.strip():
            print("输入为空，已退出。")
            break
        # 构造输入
        input_data = {
            "question": question,
            "user_preference": format_user_preference(user_profile)
        }
        # 推荐
        response = review_chain.invoke(input_data)
        # 保存上下文
        memory.save_context({"input": question}, {"output": response})
        print("\n=== 推荐结果 ===")
        print(response)
        again = input("\n是否继续提问？(y继续，其他退出)：")
        if again.lower() != "y":
            break

# === 完整流程示例 ===
if __name__ == "__main__":
    main()

欢迎使用智能美食推荐系统！
请根据提示输入你的用餐偏好（1-5分，5分最重视）
【你的用户画像已生成】
 {'环境': 5, '口味': 5, '服务': 5, '性价比': 5, '卫生': 5, '营养健康': 5, '排队时间': 5, '距离': 5, '偏好菜系': [], '不喜欢的菜系': [], '预算范围': [0, 999], '特殊要求': '无'}

请输入你的本次用餐需求（如‘预算50元内，想吃辣的，适合2人，距离不远’）：

=== 推荐结果 ===
以下是根据您的实时需求“想吃四川菜”和您的个人偏好（所有维度均为5分高要求，预算范围0-999元）推荐的餐厅。我从数据库中筛选出类型为“四川菜”或明确川菜特色的餐厅，优先匹配您的高权重指标（环境、口味、服务等所有维度均5分）。由于您未提供当前出发位置，我无法计算精确距离和交通时间（距离维度得分无法评估），推荐中会估算大致距离（基于餐厅位置描述），并在末尾建议补充位置以优化推荐。

推荐策略：
- **优先高匹配度**：所有推荐餐厅均为正宗四川菜系，避免不喜欢的菜系（无限制）。
- **权重分配**：您的偏好所有维度均5分，因此我赋予环境、口味、服务、性价比、卫生、营养健康、排队时间同等高权重。餐厅评分基于大众点评数据（dp_taste_rating等），缺失维度（如卫生、营养健康）从评论关键词和标签推断，若数据不足则标注“暂无数据”。
- **综合得分计算**：综合得分（0-100分）基于各维度加权平均（权重均为1），转换公式：`(平均维度分 / 5) * 100`。维度分使用dp评分（0-5分），缺失时用整体rating或评论推断。
- **排序**：按综合得分降序排列，推荐3家最优餐厅（得分相近时优先环境、口味高分）。
- **预算处理**：所有餐厅人均消费在您的预算内（0-999元）。
- **数据说明**：餐厅信息来自数据库，部分字段（如电话、精确距离）缺失，已标注；交通方案需您补充位置后生成。

### 推荐餐厅列表（综合得分降序）
#### 1. **火山口川味排档**
   - **基础信息**：
     - 名称：火山口川味排档
     - 地址：青岛路33-5号
     - 类型/菜系：四川菜（川菜）
     - 标签/特色：辣子鸡, 酸菜鱼, 自贡菜, 麻婆豆腐（

### 📋 历史行为学习与复用提示词
你曾收藏过部分餐厅，本次是否优先推荐类似风格？如需排除某类型/菜系请说明。


In [ ]:
# 示例：经典演示历史对话功能
# 第一次提问：询问是否有四川的面馆
question1 = "有没有四川的面馆"
response1 = review_chain.invoke({"question": question1, "user_preference": user_preference})
memory.save_context({"input": question1}, {"output": response1})
print(f"用户提问: {question1}\n助手回答: {response1}\n")

# 第二次提问：基于历史对话，进一步询问推荐
question2 = "推荐一家评分最高的四川面馆"
response2 = review_chain.invoke({"question": question2, "user_preference": user_preference})
memory.save_context({"input": question2}, {"output": response2})
print(f"用户提问: {question2}\n助手回答: {response2}\n")

# 第三次提问：基于历史对话，询问营业时间
question3 = "这家面馆今天几点开门？"
response3 = review_chain.invoke({"question": question3, "user_preference": user_preference})
memory.save_context({"input": question3}, {"output": response3})
print(f"用户提问: {question3}\n助手回答: {response3}\n")